In [1]:
import pandas as pd
import sklearn.metrics as sk_metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import random_split

from nam.wrapper import NAMClassifier, MultiTaskNAMClassifier

/home/lemeln/anaconda3/envs/nam37/lib/python3.7/site-packages/torch-1.7.0-py3.7-linux-x86_64.egg/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
# def load_compas_data():
dataset = pd.read_csv('~/data/nam/recid.data', delimiter=' ', header=None)
dataset.columns = ["age", "race", "sex", "priors_count", "length_of_stay", "c_charge_degree", "two_year_recid"]

In [3]:
dataset.head()

,age,race,sex,priors_count,length_of_stay,c_charge_degree,two_year_recid
0,69,6,2,0,1,1,0
1,34,1,2,0,10,1,1
2,24,1,2,4,1,1,1
3,44,6,2,0,1,2,0
4,41,3,2,14,6,1,1


In [4]:
# cat_ohe_step = ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
# cat_pipe = Pipeline([cat_ohe_step])
# num_pipe = Pipeline([('identity', FunctionTransformer(validate=True))])
# transformers = [('cat', cat_pipe, categorical_cols), ('num', num_pipe, numerical_cols)]
# column_transform = ColumnTransformer(transformers=transformers)

In [5]:
# categorical = ['race', 'sex', 'c_charge_degree']
binary = ['sex', 'c_charge_degree']
numerical = ['age', 'race', 'priors_count', 'length_of_stay']

In [6]:
scaler = MinMaxScaler((-1, 1))
dataset[numerical] = scaler.fit_transform(dataset[numerical])
dataset[binary] = dataset[binary] - 1

In [7]:
dataset

,age,race,sex,priors_count,length_of_stay,c_charge_degree,two_year_recid
0,0.307692,1.0,1,-1.000000,-0.9975,0,0
1,-0.589744,-1.0,1,-1.000000,-0.9750,0,1
2,-0.846154,-1.0,1,-0.789474,-0.9975,0,1
3,-0.333333,1.0,1,-1.000000,-0.9975,1,0
4,-0.410256,-0.2,1,-0.263158,-0.9850,0,1
...,...,...,...,...,...,...,...
6167,-0.871795,-1.0,1,-1.000000,-0.9950,0,0
6168,-0.871795,-1.0,1,-1.000000,-0.9950,0,0
6169,0.000000,1.0,1,-1.000000,-0.9975,0,0
6170,-0.615385,-1.0,0,-0.842105,-0.9975,1,0


In [8]:
data_train, data_test = train_test_split(dataset, train_size=0.8, test_size=0.2)
X_train, y_train = data_train[numerical + binary], data_train['two_year_recid']
X_test, y_test = data_test[numerical + binary], data_test['two_year_recid']

In [ ]:
model = NAMClassifier(
            # feature_dropout=0.0,
            # num_workers=0,
            # device='cuda',
            # val_split=0.1
            num_epochs=1000,
            num_learners=20,
            metric='auroc',
            early_stop_mode='max',
            monitor_loss=False,
            n_jobs=2
        )

model.fit(X_train.to_numpy(), y_train.to_numpy())

pred = model.predict_proba(X_test.to_numpy())
print(sk_metrics.roc_auc_score(y_test.to_numpy(), pred))

torch.FloatTensor


In [10]:
data_train.shape, data_test.shape

((4937, 7), (1235, 7))

In [11]:
X_train.dtypes

age                float64
race               float64
priors_count       float64
length_of_stay     float64
sex                  int64
c_charge_degree      int64
dtype: object